# MGTA 415 - Session 0 Demo: A simple example of paper trading with Alpaca API

In [ ]:
import alpaca_trade_api as tradeapi
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import asyncio
import websockets
import json
import os

# Alpaca API setup
alpaca_api_key = os.environ.get('ALPACA_API_KEY')
alpaca_secret_key = os.environ.get('ALPACA_SECRET_KEY')
base_url = 'https://paper-api.alpaca.markets/v2'

alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, base_url, api_version='v2')

# Sentiment analyzer
sentiment_analyzer = SentimentIntensityAnalyzer()

# WebSocket URL for real-time news
news_stream_url = "wss://stream.data.alpaca.markets/v1beta1/news"

In [ ]:
async def news_handler():
    async with websockets.connect(news_stream_url) as websocket:
        # Authenticate
        auth_data = {
            "action": "auth",
            "key": alpaca_api_key,
            "secret": alpaca_secret_key
        }
        await websocket.send(json.dumps(auth_data))
        
        # Subscribe to news
        subscribe_data = {
            "action": "subscribe",
            "news": ["*"]
        }
        await websocket.send(json.dumps(subscribe_data))
        
        while True:
            response = await websocket.recv()
            data = json.loads(response)
            if data.get('T') == 'n':  # Check if the message type is 'n' for news
                analyze_and_trade(data)

def analyze_and_trade(data):
    headline = data['headline']
    summary = data.get('summary', '')
    content = data.get('content', '')
    combined_text = f"{headline} {summary} {content}"
    
    sentiment = sentiment_analyzer.polarity_scores(combined_text)
    sentiment_score = sentiment['compound']
    
    print(f"Headline: {headline}")
    print(f"Sentiment Score: {sentiment_score}")
    
    if sentiment_score > 0.05:
        make_trade_decision('buy')
    elif sentiment_score < -0.05:
        make_trade_decision('sell')

def make_trade_decision(action):
    stock = 'AAPL'  # Example stock
    qty = 10  # Example quantity
    
    try:
        if action == 'buy':
            alpaca.submit_order(
                symbol=stock,
                qty=qty,
                side='buy',
                type='market',
                time_in_force='gtc'
            )
            print(f"Buying {qty} shares of {stock}")
        elif action == 'sell':
            alpaca.submit_order(
                symbol=stock,
                qty=qty,
                side='sell',
                type='market',
                time_in_force='gtc'
            )
            print(f"Selling {qty} shares of {stock}")
    except Exception as e:
        print(f"Error placing order: {e}")


In [ ]:
# Run the WebSocket news handler
asyncio.get_event_loop().run_until_complete(news_handler())